In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
# %config Completer.use_jedi = False

In [ ]:
import numpy as np
from pvlib import pvsystem, singlediode
import sys
module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
sys.path.insert(0, module_path)
from ipv_workbench.utilities import utils, time_utils
from ipv_workbench.translators import panelizer
from ipv_workbench.devices import devices
from ipv_workbench.solver import calculations as ipv_calc
import time
import pandas as pd

In [ ]:
project_folder = "/Users/jmccarty/Desktop/bano_simulations"
pickle_file = "/Users/jmccarty/.anydesk/incoming/2023-02-27 12:32:38.730/DP_solar_glass_B1393_raw.pickle"
cell_technology = "D"
orientation = "P"
panelizer_object = panelizer.PanelizedObject(project_folder, "B1393", pickle_file)
panelizer_object.analysis_location = 'zurich'
panelizer_object.analysis_year = 2020
panelizer_object.set_tmy_data()

# setting the analysis_period
panelizer_object.hourly_resolution = 2
panelizer_object.set_analysis_period()

custom_device_data = pd.read_csv(panelizer_object.module_cell_data, index_col='scenario').loc[
    f"{cell_technology}{orientation}"].to_dict()
panelizer_object.cell = devices.Cell(custom_device_data)